In [12]:
import torch
import torchvision.transforms as transforms
from LookGenerator.datasets.tps_dataset import ShirtsDataset
from torch.utils.data import DataLoader
from LookGenerator.networks.bpgm.model.models import BPGM
from LookGenerator.networks.bpgm.train_bpgm import train_bpgm
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

In [2]:
transform = transforms.Compose([
    transforms.Resize((256,192)),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])
mask_transform = transforms.Compose([
    transforms.Resize((256,192)),
])
batch_size = 8

In [3]:
dataset = ShirtsDataset(
    root=r"C:\Users\Даша\кто\мусор\zalando-hd-resized\train",
    transform=transform,
    transform_mask=mask_transform
)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [4]:
for data in dataloader:
    break

In [5]:
model = BPGM()

initialization method [normal]
initialization method [normal]


In [6]:
loss = train_bpgm(dataloader=dataloader,
                  model=model)

C:\Users\Даша\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Даша\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 1456/1456 [3:06:00<00:00,  7.67s/it] 


AttributeError: 'Tensor' object has no attribute 'append'

# тест

In [8]:
test_dataset = ShirtsDataset(
    root=r"C:\Users\Даша\кто\мусор\zalando-hd-resized\test",
    transform=transform,
    transform_mask=mask_transform
)

In [9]:
topil = transforms.ToPILImage()

In [23]:
testdataloader = DataLoader(test_dataset, batch_size=1)

In [24]:
for data in testdataloader:
    mask = data['segmentation']
    shirt = data['shirt']
    theta = model(mask, shirt)

    warped = F.grid_sample(shirt, theta, padding_mode='border', align_corners=True)

    warped = topil(warped[0]/2+0.5)
    warped.show()
    break